In [56]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import joblib

In [57]:
ship_data = pd.read_excel('ship_data.xlsx')
truck_data = pd.read_excel('truck_data.xlsx')

In [58]:
ship_data['입항일시'] = pd.to_datetime(ship_data['입항일시'])
ship_data['출항일시'] = pd.to_datetime(ship_data['출항일시'])
truck_data['입문시각'] = pd.to_datetime(truck_data['입문시각'])
truck_data['출문시각'] = pd.to_datetime(truck_data['출문시각'])

# 사용할 열 
ship_data = ship_data[['입항일시', '출항일시', '계선장소', 'number']]
truck_data = truck_data[['입문시각', '출문시각', '차종']]

In [59]:
# 배의 입항시간과 출항시간 사이에 있는 화물차 데이터 병합 및 선박 갯수 계산
def merge_data_with_ship_count(ship_data, truck_data):
    merged_data = []
    for _, ship_row in ship_data.iterrows():
        relevant_trucks = truck_data[
            (truck_data['입문시각'] >= ship_row['입항일시']) &
            (truck_data['출문시각'] <= ship_row['출항일시'])
        ]
        
        # 이건 선박데이터를 기준으로 아직 선박 갯수를 세는거라 여러번 선박에 대해 똑같은 결과 나올수도 있음
        # ship_count = ((ship_data['입항일시'] <= ship_row['입항일시']) & (ship_data['출항일시'] >= ship_row['출항일시'])).sum()
        
        #이건 트럭 입문시간 기준으로 입항은 되어있는데 아직 출항하지 않은 선박갯수 세는거라 정확
        ship_count = relevant_trucks.shape[0]
        
        for _, truck_row in relevant_trucks.iterrows():
            merged_data.append([
                truck_row['입문시각'].year, truck_row['입문시각'].month, truck_row['입문시각'].day,
                truck_row['입문시각'].hour,
                truck_row['차종'],
                ship_count,
                (truck_row['출문시각'] - truck_row['입문시각']).total_seconds() / 60  # 분 단위로 시간 계산
            ])
    return pd.DataFrame(merged_data, columns=[
        '입문시각_연도', '입문시각_월', '입문시각_일', '입문시각_시간', 
        '차종', '선박_갯수', '걸린시간'
    ])


In [60]:
merged_data = merge_data_with_ship_count(ship_data, truck_data)
print(merged_data.head(10)) #제대로 나오느닞 보기

   입문시각_연도  입문시각_월  입문시각_일  입문시각_시간           차종  선박_갯수   걸린시간
0     2022       1      21       11  화물 대형(5t이상)      8   82.0
1     2022       1      21        5  화물 대형(5t이상)      8  105.0
2     2022       1      21       10  화물 대형(5t이상)      8   45.0
3     2022       1      21        9  화물 대형(5t이상)      8   19.0
4     2022       1      21        9  화물 대형(5t이상)      8   25.0
5     2022       1      21        6  화물 대형(5t이상)      8   77.0
6     2022       1      21        6  화물 대형(5t이상)      8   50.0
7     2022       1      21        9  화물 대형(5t이상)      8   47.0
8     2022       1      21        8  화물 대형(5t이상)      8   47.0
9     2022       1      21        6  화물 대형(5t이상)      8   83.0


In [61]:
# 데이터 문자열로 변환
merged_data['차종'] = merged_data['차종'].astype(str)

# 데이터 분리
X = merged_data.drop(['걸린시간'], axis=1)
y = merged_data['걸린시간']

In [62]:
#레이블 인코딩
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    X[column] = label_encoders[column].fit_transform(X[column])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [63]:
# RandomForest 모델 학습
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=20, random_state=42)

In [64]:
# 모델 저장
joblib.dump(rf_model, 'random_Forest_model.pkl')

['random_Forest_model.pkl']

In [97]:
# 예측 함수
def predict_processing_time(model, year, month, day, hour, truck_type, ship_count):
    input_data = pd.DataFrame([{
        '입문시각_연도': year,
        '입문시각_월': month,
        '입문시각_일': day,
        '입문시각_시간': hour,
        '차종': truck_type,
        '선박_갯수': ship_count
    }])
    
    for column in input_data.select_dtypes(include=['object']).columns:
        input_data[column] = label_encoders[column].transform(input_data[column])
    
    predicted_time = model.predict(input_data)
    return predicted_time[0]

순서대로 년도,월,일,시간 , 차종 , 배의 갯수 입력받음

년도 < 자동입력 
시간 < 0 부터 24까지 자동입력 후 필요한 시간대만 뽑아쓰던지, 전부 뽑아서 그래프? 다른 방향으로도 필요시 코드 수정 가능

차종< 

In [95]:
def truck_size(truck_type):
    if truck_type in ["컨테이너차량", "특수차량(중장비)", "화물 대형(5t이상)"]:
        return "대"
    elif truck_type in ["화물 중형(1t이상~5t미만)"]:
        return "중"
    elif truck_type in ["화물 소형(1t미만)"]:
        return "소"
    else :
        return "알수없음"

In [102]:
# 모델 로드 및 예시로 하나 해보기
loaded_model = joblib.load('random_Forest_model.pkl')
predicted_time = predict_processing_time(loaded_model, 2024, 7, 1, 1, '화물 대형(5t이상)', 1)

print(f"예상 입출문 소요시간: 약 {predicted_time} 분" , f", 차량 크기: {truck_size}")

예상 입출문 소요시간: 약 28.35 분 , 차량 크기: <function truck_size at 0x0000013EFEAC8B80>
